
# 📑 Day 2 – 策略原型开发

## 🎯 目标与里程碑

本日的目标是：

* 使用 **Pandas-TA** 完成双均线信号（SMA20/50）+ 波动率过滤（ATR14 & z-score）；
* 基于 Pandas 实现 **向量化回测**，做到 **单日信号覆盖率 100%**；
* 完成绩效评估（年化收益、夏普比率、最大回撤）；
* 对代码进行 **函数化重构** 并配合 **pytest 单元测试**。

## 🕘 回顾 Day1

* 整理了 **数据管道与EDA** 代码：包括 yfinance 批量拉取、保存为 Parquet、交互式 K 线图。
* 确认数据结构：索引为日期，列为 `Open / High / Low / Close / Volume`。
* **改进：路径管理**：引入 `pathlib` 统一处理文件路径，自动识别当前脚本所在目录（或 Notebook 工作目录），并在其下创建 `data/` 子目录，用于存放输出的 Parquet 文件，保证跨平台兼容性和可维护性。


In [70]:
# 安装依赖

# %pip install ta


## 1. 指标计算（`calculate_indicators`）

### 1.1 SMA 20/50

* 使用 `ta.trend.SMAIndicator` 计算 20 日、50 日的收盘价均线。
* 分别生成 `SMA_20` 和 `SMA_50` 列。

```python
df['SMA_20'] = SMAIndicator(close=df.Close, window=20).sma_indicator()
df['SMA_50'] = SMAIndicator(close=df.Close, window=50).sma_indicator()
```

### 1.2 ATR 14

* 使用 `ta.volatility.AverageTrueRange` 计算 14 日 **平均真实波幅 (ATR)**，衡量市场波动率。
* 生成 `ATR_14` 列。

```python
df['ATR_14'] = AverageTrueRange(high=df.High, low=df.Low, close=df.Close, window=14).average_true_range()
```

### 1.3 波动率过滤 z-score

* 用 ATR 的 **252 日滚动均值与标准差**计算 z-score，识别异常波动时期。
* 生成 `zscore` 列。

```python
df['zscore'] = (df['ATR_14'] - df['ATR_14'].rolling(252).mean()) / df['ATR_14'].rolling(252).std()
```

### 关键概念

* **向量化计算**：利用 `ta` 库的指标，避免循环，速度更快。
* **滚动窗口**：`rolling(N)` 计算 N 日滑动均值/标准差，适合时序指标。
* **z-score 标准化**：将波动率转换为标准差单位，便于识别异常情况。


In [71]:
from ta.trend import SMAIndicator
from ta.volatility import AverageTrueRange
from ta.utils import dropna

import pandas as pd
import numpy as np
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
import os


# 定义保存指标到parquet的函数
def save_backtest_result_to_parquet(ticker, df, output_dir="data/backtest_results"):
    # 只保留关键指标列
    metrics_cols = ['SMA_20', 'SMA_50', 'ATR_14', 'Signal', 'Vol_filter', 'Position', 'Return', 'Strategy_Return', 'Equity_Curve']
    # 过滤存在的列
    cols_to_save = [col for col in metrics_cols if col in df.columns]
    metrics_df = df[cols_to_save].copy()
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    # 保存为parquet文件
    pq.write_table(pa.Table.from_pandas(metrics_df), os.path.join(output_dir, f"{ticker}_backtest.parquet"))


In [72]:


# 计算指标函数
def calculate_indicators(df):
    
    # 1.1 计算 SMA 20/50：简单移动平均线，分别取20日和50日收盘价均值。
    df['SMA_20'] = SMAIndicator(close=df.Close, window=20).sma_indicator()
    df['SMA_50'] = SMAIndicator(close=df.Close, window=50).sma_indicator()

    # 1.2 计算 ATR(14)：平均真实波幅（衡量波动率）
    df['ATR_14'] = AverageTrueRange(
        high=df.High,
        low=df.Low,
        close=df.Close,
        window=14
    ).average_true_range()

    # 1.3 用 ATR 的 z-score（标准化）作为过滤条件。
    df['zscore'] = (df['ATR_14'] - df['ATR_14'].rolling(252).mean()) / df['ATR_14'].rolling(252).std()

    return df



## 2. 策略逻辑（`strategy_logic`）

### 2.1 双均线交叉信号

* 当 `SMA_20 > SMA_50` 时产生买入信号，否则为空仓。
* 生成 `Signal` 列。

```python
df['Signal'] = np.where(df['SMA_20'] > df['SMA_50'], 1, 0)
```

### 2.2 波动率过滤

* 当 ATR z-score < 1（即波动率较低）时才允许开仓。
* 生成 `Vol_filter` 列。

```python
df['Vol_filter'] = df['zscore'] < 1
```

In [73]:

# 策略逻辑函数
def strategy_logic(df):
    
    # 2.1 加载策略逻辑，生成 signal 列（1=多仓, 0=空仓）
    df['Signal'] = np.where(df['SMA_20'] > df['SMA_50'], 1, 0) # 金叉出现
    df['Vol_filter'] = df['zscore'] < 1  # 波动率低于1倍标准差时允许开仓

    return df



## 3. 持仓矩阵（`position_matrix`）

### 3.1 滞后处理

* 持仓信号需要滞后 1 天，避免“未来函数”。
* 初始持仓为 0，并用前向填充保持连续。

```python
df['Position'] = (df['Signal'] & df['Vol_filter']).shift(1)
df['Position'] = df['Signal'].shift(1).ffill().fillna(0)
```

> ⚠️ 注意：在当前代码里，第二行会覆盖掉 `Vol_filter`，实际执行的是**纯双均线策略**。如果想保留波动率过滤，应去掉第二行，或者改为：

```python
df['Position'] = (df['Signal'].astype(bool) & df['Vol_filter']).shift(1).ffill().fillna(False).astype(int)
```


In [74]:

# 生成持仓矩阵
def position_matrix(df):

    # 3.1 生成持仓矩阵： 信号和波动率过滤都滞后1天，模拟真实持仓决策
    df['Position'] = (
        (df['Signal'].astype(bool) & df['Vol_filter'])
        .shift(1)
        .astype('boolean')   
        .ffill()
        .fillna(False)
        .astype(int) 
    )

    return df


## 4. 回测计算（`calculate_returns`）

### 4.1 日收益率与策略收益

* `Return`：标的每日收益率 = `Close` 的对数收益 / pct\_change。
* `Strategy_Return`：策略收益 = 当日持仓 \* 标的收益率。

```python
df['Return'] = df['Close'].pct_change()
df['Strategy_Return'] = df['Position'] * df['Return']
```

### 4.2 策略净值曲线

* 用累计乘积计算净值曲线。

```python
df['Equity_Curve'] = (1 + df['Strategy_Return']).cumprod()
```


In [75]:


# 回测：收益率计算
def calculate_returns(df):

    # 4.1 盈亏计算
    df['Return'] = df['Close'].pct_change() # 市场收益率
    df['Strategy_Return'] = df['Position'] * df['Return'] # 策略收益率 = 持仓 * 市场收益率
    df['Equity_Curve'] = (1 + df['Strategy_Return']).cumprod() # 累积计算收益

    return df



## 5. 策略评估（`evaluate_strategy`）

针对每个标的，计算核心绩效指标：

* **CAGR**（年化复合收益率）
* **AnnReturn**（年化收益率）
* **AnnVol**（年化波动率）
* **Sharpe**（夏普比率）
* **MaxDrawdown**（最大回撤）及对应时间区间
* **TotalReturn**（总收益）
* **N**（样本点数）

返回结果是一个字典，例如：

```python
{
 'CAGR': 0.136, 
 'AnnReturn': 0.150, 
 'AnnVol': 0.212, 
 'Sharpe': 0.708, 
 'MaxDrawdown': -0.291, 
 'MDD_Peak': Timestamp('2022-01-03'), 
 'MDD_Trough': Timestamp('2023-03-01'), 
 'TotalReturn': 1.144, 
 'N': 1507
}
```


In [76]:

# 策略评估
def evaluate_strategy(df, periods_per_year: int = 252, risk_free_rate_annual: float = 0.0):

    # 提取必要的列
    er = df['Equity_Curve'].dropna().astype(float)
    r  = df['Strategy_Return'].replace([np.inf, -np.inf], np.nan).dropna().astype(float)

    #计算 CAGR, 年化收益率, 年化波动率, 夏普比率
    if er.empty or er.iloc[0] <= 0:
        cagr, total_return, years = np.nan, np.nan, np.nan
    else:
        total_return = er.iloc[-1] - 1.0
        if isinstance(er.index, pd.DatetimeIndex) and len(er) > 1:
            years = (er.index[-1] - er.index[0]).days / 365.25
        else:
            years = len(er) / periods_per_year
        cagr = er.iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan

    #年化收益 & 夏普
    if r.empty:
        ann_ret, ann_vol, sharpe, n_obs = np.nan, np.nan, np.nan, 0
    else:
        n_obs = len(r)
        ann_ret = r.mean() * periods_per_year
        ann_vol = r.std(ddof=0) * np.sqrt(periods_per_year)
        sharpe = (ann_ret - risk_free_rate_annual) / ann_vol if ann_vol > 0 else np.nan

    #最大回撤
    if er.empty:
        mdd, peak, trough = np.nan, None, None
    else:
        roll_max = er.cummax()
        dd = er / roll_max - 1
        trough = dd.idxmin()
        peak = er.loc[:trough].idxmax() if trough is not None else None
        mdd = dd.min()

    return {
        'CAGR': float(cagr) if pd.notna(cagr) else np.nan,
        'AnnReturn': float(ann_ret) if pd.notna(ann_ret) else np.nan,
        'AnnVol': float(ann_vol) if pd.notna(ann_vol) else np.nan,
        'Sharpe': float(sharpe) if pd.notna(sharpe) else np.nan,
        'MaxDrawdown': float(mdd) if pd.notna(mdd) else np.nan,
        'MDD_Peak': peak,
        'MDD_Trough': trough,
        'TotalReturn': float(total_return) if pd.notna(total_return) else np.nan,
        'N': n_obs
    }



## 主函数：



### 关键概念：

1. **年化收益率 (CAGR)**

   $$
   CAGR = \left(\frac{Equity_{end}}{Equity_{start}}\right)^{\frac{252}{n}} - 1
   $$

2. **夏普比率 (Sharpe Ratio)**

   $$
   Sharpe = \frac{E[daily\_return]}{Std[daily\_return]} \times \sqrt{252}
   $$

3. **最大回撤 (Max Drawdown)**

   * 基于 `equity_curve` 的滚动高点计算；
   * `drawdown = equity_curve / equity_curve.cummax() - 1`。


In [77]:

spy_top10 = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOGL', 'META', 'BRK-B', 'AVGO', 'TSLA', 'GOOG']
base_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
parquet_dir = base_dir / "data"

for ticker in spy_top10:
    parquet_path = os.path.join(parquet_dir, f"{ticker}.parquet")
    
    if os.path.exists(parquet_path):

        # 读取 Parquet 文件
        df = dropna(pd.read_parquet(parquet_path))

        # 计算指标
        df = calculate_indicators(df)

        # 应用策略逻辑
        df = strategy_logic(df)

        # 生成持仓矩阵
        df = position_matrix(df)

        # 计算收益率
        df = calculate_returns(df)

        # 评估策略
        metrics = evaluate_strategy(df)

        # 在每次计算完毕后保存
        save_backtest_result_to_parquet(ticker, df)

        # 打印 ticker 名称
        print(f"Ticker: {ticker}")
    
        # 检查 signal 覆盖率
        coverage = 1 - (df['Signal'].isna().sum() / len(df))
        print(f"Signal Coverage: {coverage:.2%}")

        # 打印评估结果
        for k, v in metrics.items():
            print(f"{k:<12}: {v}")

        print("-" * 40)

    else:
        print(f"Parquet file for {ticker} does not exist: {parquet_path}")
        continue



Ticker: AAPL
Signal Coverage: 100.00%
CAGR        : 0.017304659211901274
AnnReturn   : 0.027616534820316025
AnnVol      : 0.14448030626818792
Sharpe      : 0.19114393880819666
MaxDrawdown : -0.3220917348538129
MDD_Peak    : 2021-01-26 00:00:00
MDD_Trough  : 2023-03-01 00:00:00
TotalReturn : 0.10834662099623205
N           : 1507
----------------------------------------
Ticker: MSFT
Signal Coverage: 100.00%
CAGR        : 0.054998228896864454
AnnReturn   : 0.06588278605853151
AnnVol      : 0.15608281514159503
Sharpe      : 0.422101472213735
MaxDrawdown : -0.3954908640144694
MDD_Peak    : 2021-09-16 00:00:00
MDD_Trough  : 2023-03-01 00:00:00
TotalReturn : 0.37852578425869976
N           : 1507
----------------------------------------
Ticker: AMZN
Signal Coverage: 100.00%
CAGR        : 0.04445883606310286
AnnReturn   : 0.06156817753081885
AnnVol      : 0.18968371313008411
Sharpe      : 0.32458336308819363
MaxDrawdown : -0.3877686686407408
MDD_Peak    : 2021-02-02 00:00:00
MDD_Trough  : 202